In [1]:
import os
import sys
import shutil
import numpy as np
sys.path.insert(0, '../scripts')

from eeg_cnn_lib import transfer_all_class, prepare_data_labels
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout
from keras import optimizers

# The images are in a folder named 'shapes/training'
train_dir_name = '../../../data/classification/valence/train'
validation_dir_name = '../../../data/classification/valence/validation'
test_dir_name = '../../../data/classification/valence/test'


# All images are 32 pixels
img_size = (32,32)
img_size_rgba = (32,32,4)

# The folder contains a subfolder for each class of shape
classes = sorted(os.listdir(train_dir_name))

Using TensorFlow backend.


In [2]:
import sys
import keras
print('Keras version:',keras.__version__)

from keras import backend as K

Keras version: 2.3.1


In [3]:
transfer_all_class(test_dir_name,train_dir_name,1.0,classes)
transfer_all_class(validation_dir_name,train_dir_name,1.0,classes)
transfer_all_class(train_dir_name,test_dir_name,0.2,classes)

In [4]:
X, Y = prepare_data_labels(classes, train_dir_name)
X=np.asarray(X)
Y=np.asarray(Y)

In [5]:
model = Sequential()

model.add(Conv2D(32, (3, 3), strides=1, padding='same', input_shape=img_size_rgba, activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2),strides=2))

model.add(Conv2D(64, (3, 3), strides=1, padding='same', input_shape=img_size_rgba, activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2),strides=2))

model.add(Conv2D(128, (3, 3), strides=1, padding='same', input_shape=img_size_rgba, activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2),strides=2))

model.add(Flatten())

model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))

model.add(Dense(len(classes), activation='softmax'))

opt = optimizers.Adam(lr=0.001)

model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        1184      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)             

In [6]:
kf = KFold(n_splits=10, shuffle=False)
accuracies = []

In [7]:
for train_index, val_index in kf.split(X, Y):
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    transfer_all_class(validation_dir_name,train_dir_name,1.0,classes)
    for index in range(len(X_val)):
        class_label=''
        if(Y_val[index]==0):
            class_label=classes[0]
        else:
            class_label=classes[1]  
        #Then, copy the validation images to the validation folder
        shutil.move(train_dir_name+"/"+class_label+'/'+X_val[index], 
                    validation_dir_name+"/"+class_label+'/'+X_val[index])
        
    batch_size = 256
    epoch=100
    datagen = ImageDataGenerator(rescale=1./255)

    train_generator = datagen.flow_from_directory(
        train_dir_name,
        target_size=img_size,
        color_mode='rgba',
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = datagen.flow_from_directory(
        validation_dir_name,
        target_size=img_size,
        color_mode='rgba',
        batch_size=batch_size,
        class_mode=None)
        
    history=model.fit_generator(train_generator, 
                        epochs=epoch)
        
    predictions = model.predict_generator(validation_generator, verbose=1)
    y_predictions = np.argmax(predictions, axis=1)
    y_true = validation_generator.classes
    accuracy=accuracy_score(y_true, y_predictions)
    accuracies.append(accuracy)
    print(accuracy)

Found 12241 images belonging to 2 classes.
Found 1361 images belonging to 2 classes.
Epoch 1/100
48/48 [==============================] - 14s 292ms/step - loss: 0.6850 - accuracy: 0.5734
Epoch 2/100
48/48 [==============================] - 8s 160ms/step - loss: 0.6807 - accuracy: 0.5814
Epoch 3/100
48/48 [==============================] - 8s 161ms/step - loss: 0.6790 - accuracy: 0.5814
Epoch 4/100
48/48 [==============================] - 8s 160ms/step - loss: 0.6787 - accuracy: 0.5814
Epoch 5/100
48/48 [==============================] - 8s 160ms/step - loss: 0.6779 - accuracy: 0.5807
Epoch 6/100
48/48 [==============================] - 8s 160ms/step - loss: 0.6758 - accuracy: 0.5814
Epoch 7/100
48/48 [==============================] - 8s 161ms/step - loss: 0.6753 - accuracy: 0.5819
Epoch 8/100
48/48 [==============================] - 8s 161ms/step - loss: 0.6742 - accuracy: 0.5872
Epoch 9/100
48/48 [==============================] - 8s 161ms/step - loss: 0.6710 - accuracy: 0.5877
Epoch

48/48 [==============================] - 8s 159ms/step - loss: 0.2339 - accuracy: 0.8935
Epoch 81/100
48/48 [==============================] - 8s 160ms/step - loss: 0.2273 - accuracy: 0.8946
Epoch 82/100
48/48 [==============================] - 8s 160ms/step - loss: 0.2177 - accuracy: 0.8985
Epoch 83/100
48/48 [==============================] - 8s 159ms/step - loss: 0.2214 - accuracy: 0.8997
Epoch 84/100
48/48 [==============================] - 8s 160ms/step - loss: 0.2151 - accuracy: 0.9036
Epoch 85/100
48/48 [==============================] - 8s 160ms/step - loss: 0.2059 - accuracy: 0.9043
Epoch 86/100
48/48 [==============================] - 8s 160ms/step - loss: 0.2158 - accuracy: 0.9019
Epoch 87/100
48/48 [==============================] - 8s 160ms/step - loss: 0.2083 - accuracy: 0.9063
Epoch 88/100
48/48 [==============================] - 8s 160ms/step - loss: 0.2009 - accuracy: 0.9101
Epoch 89/100
48/48 [==============================] - 8s 159ms/step - loss: 0.1950 - accuracy: 

Epoch 59/100
48/48 [==============================] - 8s 159ms/step - loss: 0.1143 - accuracy: 0.9521
Epoch 60/100
48/48 [==============================] - 8s 159ms/step - loss: 0.1239 - accuracy: 0.9450
Epoch 61/100
48/48 [==============================] - 8s 159ms/step - loss: 0.1198 - accuracy: 0.9480
Epoch 62/100
48/48 [==============================] - 8s 158ms/step - loss: 0.1218 - accuracy: 0.9476
Epoch 63/100
48/48 [==============================] - 8s 158ms/step - loss: 0.1195 - accuracy: 0.9484
Epoch 64/100
48/48 [==============================] - 8s 159ms/step - loss: 0.1190 - accuracy: 0.9476
Epoch 65/100
48/48 [==============================] - 8s 159ms/step - loss: 0.1221 - accuracy: 0.9468
Epoch 66/100
48/48 [==============================] - 8s 158ms/step - loss: 0.1163 - accuracy: 0.9498
Epoch 67/100
48/48 [==============================] - 8s 159ms/step - loss: 0.1090 - accuracy: 0.9537
Epoch 68/100
48/48 [==============================] - 8s 158ms/step - loss: 0.1062

48/48 [==============================] - 8s 177ms/step - loss: 0.1004 - accuracy: 0.9589
Epoch 38/100
48/48 [==============================] - 8s 168ms/step - loss: 0.0823 - accuracy: 0.9652
Epoch 39/100
48/48 [==============================] - 8s 162ms/step - loss: 0.0949 - accuracy: 0.9596
Epoch 40/100
48/48 [==============================] - 8s 161ms/step - loss: 0.0971 - accuracy: 0.9598
Epoch 41/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0889 - accuracy: 0.9633
Epoch 42/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0854 - accuracy: 0.9632
Epoch 43/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0866 - accuracy: 0.9624
Epoch 44/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0897 - accuracy: 0.9627
Epoch 45/100
48/48 [==============================] - 8s 161ms/step - loss: 0.0904 - accuracy: 0.9616
Epoch 46/100
48/48 [==============================] - 8s 164ms/step - loss: 0.0826 - accuracy: 

Epoch 16/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0852 - accuracy: 0.9647
Epoch 17/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0855 - accuracy: 0.9654
Epoch 18/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0735 - accuracy: 0.9697
Epoch 19/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0720 - accuracy: 0.9723
Epoch 20/100
48/48 [==============================] - 8s 176ms/step - loss: 0.0783 - accuracy: 0.9678
Epoch 21/100
48/48 [==============================] - 10s 199ms/step - loss: 0.0789 - accuracy: 0.9698
Epoch 22/100
48/48 [==============================] - 10s 202ms/step - loss: 0.0779 - accuracy: 0.9685
Epoch 23/100
48/48 [==============================] - 10s 202ms/step - loss: 0.0690 - accuracy: 0.9714s - l
Epoch 24/100
48/48 [==============================] - 10s 201ms/step - loss: 0.0667 - accuracy: 0.9746
Epoch 25/100
48/48 [==============================] - 10s 207ms/step - lo

...

48/48 [==============================] - 9s 193ms/step - loss: 0.0419 - accuracy: 0.9848
Epoch 31/100
48/48 [==============================] - 9s 179ms/step - loss: 0.0478 - accuracy: 0.9828
Epoch 32/100
48/48 [==============================] - 8s 170ms/step - loss: 0.0458 - accuracy: 0.9828
Epoch 33/100
48/48 [==============================] - 8s 164ms/step - loss: 0.0473 - accuracy: 0.9822
Epoch 34/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0435 - accuracy: 0.9833
Epoch 35/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0467 - accuracy: 0.9824
Epoch 36/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0457 - accuracy: 0.9828
Epoch 37/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0381 - accuracy: 0.9851
Epoch 38/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0405 - accuracy: 0.9854
Epoch 39/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0444 - accuracy: 

Epoch 9/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0464 - accuracy: 0.9837
Epoch 10/100
48/48 [==============================] - 8s 162ms/step - loss: 0.0515 - accuracy: 0.9804
Epoch 11/100
48/48 [==============================] - 8s 162ms/step - loss: 0.0467 - accuracy: 0.9835
Epoch 12/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0504 - accuracy: 0.9824
Epoch 13/100
48/48 [==============================] - 8s 162ms/step - loss: 0.0564 - accuracy: 0.9800
Epoch 14/100
48/48 [==============================] - 8s 161ms/step - loss: 0.0490 - accuracy: 0.9808
Epoch 15/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0490 - accuracy: 0.9815
Epoch 16/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0464 - accuracy: 0.9835
Epoch 17/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0499 - accuracy: 0.9828
Epoch 18/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0401 

48/48 [==============================] - 8s 160ms/step - loss: 0.0356 - accuracy: 0.9869
Epoch 90/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0344 - accuracy: 0.9888
Epoch 91/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0373 - accuracy: 0.9875
Epoch 92/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0394 - accuracy: 0.9860
Epoch 93/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0318 - accuracy: 0.9874
Epoch 94/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0399 - accuracy: 0.9860
Epoch 95/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0432 - accuracy: 0.9840
Epoch 96/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0365 - accuracy: 0.9846
Epoch 97/100
48/48 [==============================] - 8s 159ms/step - loss: 0.0346 - accuracy: 0.9869
Epoch 98/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0362 - accuracy: 

Epoch 68/100
48/48 [==============================] - 11s 239ms/step - loss: 0.0388 - accuracy: 0.9853
Epoch 69/100
48/48 [==============================] - 10s 207ms/step - loss: 0.0337 - accuracy: 0.9876
Epoch 70/100
48/48 [==============================] - 9s 188ms/step - loss: 0.0409 - accuracy: 0.9844
Epoch 71/100
48/48 [==============================] - 8s 168ms/step - loss: 0.0365 - accuracy: 0.9875
Epoch 72/100
48/48 [==============================] - 8s 162ms/step - loss: 0.0341 - accuracy: 0.9863
Epoch 73/100
48/48 [==============================] - 8s 161ms/step - loss: 0.0404 - accuracy: 0.9861
Epoch 74/100
48/48 [==============================] - 8s 160ms/step - loss: 0.0366 - accuracy: 0.9868
Epoch 75/100
48/48 [==============================] - 8s 161ms/step - loss: 0.0376 - accuracy: 0.9866
Epoch 76/100
48/48 [==============================] - 8s 168ms/step - loss: 0.0399 - accuracy: 0.9842
Epoch 77/100
48/48 [==============================] - 8s 168ms/step - loss: 0.03

48/48 [==============================] - 8s 164ms/step - loss: 0.0353 - accuracy: 0.9873
Epoch 47/100
48/48 [==============================] - 8s 166ms/step - loss: 0.0334 - accuracy: 0.9882
Epoch 48/100
48/48 [==============================] - 8s 170ms/step - loss: 0.0422 - accuracy: 0.9853
Epoch 49/100
48/48 [==============================] - 8s 176ms/step - loss: 0.0321 - accuracy: 0.9892
Epoch 50/100
48/48 [==============================] - 8s 168ms/step - loss: 0.0320 - accuracy: 0.9879
Epoch 51/100
48/48 [==============================] - 8s 169ms/step - loss: 0.0376 - accuracy: 0.9870
Epoch 52/100
48/48 [==============================] - 8s 169ms/step - loss: 0.0374 - accuracy: 0.9855
Epoch 53/100
48/48 [==============================] - 8s 169ms/step - loss: 0.0386 - accuracy: 0.9856
Epoch 54/100
48/48 [==============================] - 9s 178ms/step - loss: 0.0352 - accuracy: 0.9881
Epoch 55/100
48/48 [==============================] - 8s 176ms/step - loss: 0.0400 - accuracy: 

In [9]:
np.average(accuracies)

0.8883504775900075

In [10]:
test_generator = datagen.flow_from_directory(
    test_dir_name,
    target_size=img_size,
    color_mode='rgba',
    batch_size=batch_size,
    class_mode='categorical')

Found 3400 images belonging to 2 classes.


In [11]:
loss, acc = model.evaluate_generator(test_generator)
acc

0.7076470851898193